In [102]:
# Dependencies and Setup
import pandas as pd
import requests
from pathlib import Path
import hvplot.pandas

In [103]:
# Save path to the data set as a variable.
data_file = Path("Resources/Natural_Disasters.csv")

# Read our data file with the Pandas library
natural_disasters_df = pd.read_csv(data_file)
natural_disasters_df

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,1,Afghanistan,AF,AFG,Drought,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,2,Afghanistan,AF,AFG,Extreme temperature,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Afghanistan,AF,AFG,Flood,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,4,Afghanistan,AF,AFG,Landslide,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,5,Afghanistan,AF,AFG,Storm,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,964,Zambia,ZM,ZMB,Flood,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,3.0,NaN,1.0
751,965,Zambia,ZM,ZMB,Landslide,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
752,967,Zimbabwe,ZW,ZWE,Drought,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
753,968,Zimbabwe,ZW,ZWE,Flood,Number,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [104]:
# Remove extraneous column
Natural_disaster_reduced_df = natural_disasters_df.drop(columns=['ISO2', 'ObjectId', 'Source', 'CTS_Code', 'CTS_Name', 'CTS_Full_Descriptor'])
Natural_disaster_reduced_df.head()

,Country,ISO3,Indicator,Unit,F1980,F1981,F1982,F1983,F1984,F1985,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,Afghanistan,AFG,Drought,Number,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,Afghanistan,AFG,Extreme temperature,Number,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Flood,Number,1.0,NaN,NaN,NaN,NaN,NaN,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,Afghanistan,AFG,Landslide,Number,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,Afghanistan,AFG,Storm,Number,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN


In [105]:
# Remove remaining rows with missing data
natural_disasters_cleaned_df = Natural_disaster_reduced_df.fillna(0)

natural_disasters_cleaned_df.head(10)

,Country,ISO3,Indicator,Unit,F1980,F1981,F1982,F1983,F1984,F1985,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,Afghanistan,AFG,Drought,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,Afghanistan,AFG,Extreme temperature,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,Flood,Number,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,Afghanistan,AFG,Landslide,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0
4,Afghanistan,AFG,Storm,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
5,Afghanistan,AFG,Wildfire,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Albania,ALB,Drought,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Albania,ALB,Extreme temperature,Number,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,Albania,ALB,Flood,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
9,Albania,ALB,Landslide,Number,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:

# rename columns:
natural_disasters_renamed_df = natural_disasters_cleaned_df.rename(columns={"ISO3":"Country_Code","Indicator":"Disasters",
                                                "F1980":"1980", "F1981":"1981", "F1982":"1982",
                                                "F1983":"1983","F1984":"1984","F1985":"1985",
                                                "F1986":"1986","F1987":"1987","F1988":"1988",
                                                "F1989":"1989","F1990":"1990","F1991":"1991","F1992":"1992",
                                                "F1993":"1993","F1994":"1994","F1995":"1995",
                                                "F1996":"1996","F1997":"1997","F1998":"1998",
                                                "F1999":"1999","F2000":"2000","F2001":"2001",
                                                "F2002":"2002","F2003":"2003","F2004":"2004",
                                                "F2005":"2005","F2006":"2006","F2007":"2007",
                                                "F2008":"2008","F2009":"2009","F2010":"2010",
                                                "F2011":"2011","F2012":"2012","F2013":"2013",
                                                "F2014":"2014","F2015":"2015","F2016":"2016",
                                                "F2017":"2017","F2018":"2018","F2019":"2019",
                                                 "F2020":"2020","F2021":"2021","F2022":"2022" })
natural_disasters_renamed_df.head(20)



,Country,Country_Code,Disasters,Unit,1980,1981,1982,1983,1984,1985,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,AFG,Drought,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,Afghanistan,AFG,Extreme temperature,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,Flood,Number,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,Afghanistan,AFG,Landslide,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0
4,Afghanistan,AFG,Storm,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
5,Afghanistan,AFG,Wildfire,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Albania,ALB,Drought,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Albania,ALB,Extreme temperature,Number,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,Albania,ALB,Flood,Number,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
9,Albania,ALB,Landslide,Number,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
